In [1]:
from tools import GPUTools

In [2]:
import numpy as np

In [3]:
numpy_array = np.array([1,2,3,4,5])

device_3 = GPUTools.to_device(numpy_array, device="gpu", target_gpu=3)
device_0 = GPUTools.to_device(numpy_array, device="gpu", target_gpu=0)

In [4]:
print(device_3.device)
print(device_0.device)

<CUDA Device 3>
<CUDA Device 0>


In [5]:
device_2 = GPUTools.to_device(device_3/device_0, device="gpu", target_gpu=2)
print(device_2.device)

/tmp/ipykernel_910/1199105160.py:1: PerformanceWarning: The device where the array resides (3) is different from the current device (0). Peer access has been activated automatically.
  device_2 = GPUTools.to_device(device_3/device_0, device="gpu", target_gpu=2)


<CUDA Device 2>


In [6]:
device_2.__module__

'cupy'

In [ ]:
chunksize = X.chunksize
chunksize_numpy = np.array((chunksize))
chunksize_numpy

In [ ]:
chunksize_numpy[[2,3,4]] = -1

In [ ]:
tuple(chunksize_numpy)

In [ ]:
len(X.shape)

In [ ]:
X.dtype

In [ ]:
np.array([1,2,3,4.5]).dtype

In [ ]:
X.ndim

In [1]:
# TEST GPU ALLOCATION BEFORE COMPUTATION FT

In [2]:
from tools import FourierTools, SpaceEstimator
import dask.array as da
import numpy as np

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

In [3]:
X_np = np.random.randn(100,100,100, 10, 10)
SpaceEstimator.for_array(X_np)

<Quantity(762.939453, 'mebibyte')>

In [4]:
X = da.asarray(X_np.astype(np.float32), chunks=(10,20,30, 40, 50))
X

dask.array<array, shape=(100, 100, 100, 10, 10), dtype=float32, chunksize=(10, 20, 30, 10, 10), chunktype=numpy.ndarray>

In [5]:
X_fft = FourierTools.cartesian_FT_dask(array=X, direction="forward", fft_shift=True, axes=(2,3,4), device="gpu")
X_fft

[  1  ][ success ] ---v  (collected several lines)                      
                      Adding Fourier Transformation to Dask Graph:
                       => Direction:............... forward
                       => Applying (i)FFT-shift:... True
                       => Affected axes:........... (2, 3, 4) of 5 total axes
                       => Rechunking:.............. (10, 20, 30, 10, 10) => (10, 20, 100, 10, 10)
                       => Data type:............... float32 => <class 'numpy.complex64'>
                       => Performing on device:.... gpu
                      


dask.array<fftshift, shape=(100, 100, 100, 10, 10), dtype=complex64, chunksize=(10, 20, 100, 10, 10), chunktype=cupy.ndarray>

In [8]:
import cupy as cp
meta = cp.empty((0,) * X_fft.ndim, dtype=X_fft.dtype)

In [11]:
meta.dtype

dtype('complex64')

In [1]:
from cupyx.scipy.ndimage import zoom
from tools import GPUTools, SpaceEstimator
import numpy as np
import cupy as cp

In [2]:
array_np = np.random.randn(100,100,100, 10, 10)
SpaceEstimator.for_array(array_np)

array = array_np

zoom_factor = np.divide((110,110,110, 10, 10), (100,100,100, 10, 10))
zoom_factor

array([1.1, 1.1, 1.1, 1. , 1. ])

In [3]:
order = 1
target_gpu=2
return_on_device="gpu"

array = GPUTools.run_cupy_local_pool(
    working_function=lambda: zoom(
        input=(array if isinstance(array, cp.ndarray) else cp.asarray(array)),
        zoom=zoom_factor,
        order=order
    ),
    target_gpu=target_gpu,
    return_device=return_on_device
)

In [4]:
array = GPUTools.to_device(array, "cpu", target_gpu=3)

UnboundLocalError: local variable 'x' referenced before assignment

In [5]:
type(array)

numpy.ndarray

In [6]:
with cp.cuda.Device(2):
    cp.cuda.get_current_stream().synchronize()
    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()  # optional, for host-pinned buffers


In [7]:
with cp.cuda.Device(2):
    _ = array.sum()

In [8]:
cp.cuda.runtime.deviceCanAccessPeer(3, array.device.id)

1

In [9]:
cp.cuda.runtime.deviceEnablePeerAccess(3)

In [11]:
GPUTools.to_device(array, "gpu", target_gpu=3).device

<CUDA Device 3>

In [ ]:
import cupy as cp
print(type(array))
if isinstance(array, cp.ndarray):
    print("src device:", array.device.id)
print("device count:", cp.cuda.runtime.getDeviceCount())

In [8]:
import cupy as cp
src = 2
dst = 3
print(cp.cuda.runtime.deviceCanAccessPeer(dst, src))  # 1 oder 0
print(cp.cuda.runtime.deviceCanAccessPeer(src, dst))  # 1 oder 0

1
1


In [9]:
src, dst = 2, 3

with cp.cuda.Device(dst):
    try:
        cp.cuda.runtime.deviceEnablePeerAccess(src)
    except cp.cuda.runtime.CUDARuntimeError:
        pass

with cp.cuda.Device(src):
    try:
        cp.cuda.runtime.deviceEnablePeerAccess(dst)
    except cp.cuda.runtime.CUDARuntimeError:
        pass

with cp.cuda.Device(dst):
    x_on_dst = cp.array(array, copy=True)  # erzwingt echte Kopie auf dst

CUDARuntimeError: cudaErrorInvalidValue: invalid argument